In [15]:
# Politican pipeline 
import plotly.plotly as py
import pandas as pd
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
import pickle
from bqplot import *

In [56]:
# set the user 
# Charlie Baker Example
#user = 'MassGovernor'
#user = 'realDonaldTrump'
#user = 'SenWarren'
user = 'BarackObama'
data_file = '{}_sentiment_data.csv'.format(user)
hash_data = '{}_hashtags.pkl'.format(user)
macro_sent_data = '{}_sent_stats.pkl'.format(user)
data_path = '../Twitter_to_vec/user_tweets/'

In [57]:
df = pd.read_csv(data_path + user + "/" + data_file)

In [58]:
df.head()

,Unnamed: 0,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,...,retweeted_status,source,text,truncated,user,preds,w_preds,pos_proba,w_pos_proba,hashtags
0,0,NaN,NaN,Mon Apr 23 19:13:36 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,190244,False,NaN,988496127996678145,...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",These talented young people will create a bett...,True,"{'id': 813286, 'id_str': '813286', 'name': 'Ba...",1,190244,0.882768,167941.280379,[]
1,1,NaN,NaN,Wed Apr 18 00:44:58 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 986405186611560448, 'id_str'...",322897,False,NaN,986405191711944704,...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Our statement on the passing of Former First L...,False,"{'id': 813286, 'id_str': '813286', 'name': 'Ba...",1,322897,0.808772,261150.161494,[]
2,2,NaN,NaN,Mon Apr 16 11:25:04 +0000 2018,"{'hashtags': [{'text': 'ObamaFellows', 'indice...",NaN,0,False,NaN,985841501321269250,...,{'created_at': 'Mon Apr 16 11:14:30 +0000 2018...,"<a href=""http://twitter.com/download/iphone"" r...",RT @ObamaFoundation: We just announced the ina...,False,"{'id': 813286, 'id_str': '813286', 'name': 'Ba...",0,0,0.478186,0.000000,['ObamaFellows']
3,3,NaN,NaN,Wed Apr 04 14:35:58 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,NaN,981540886726217729,...,{'created_at': 'Wed Apr 04 14:29:58 +0000 2018...,"<a href=""http://twitter.com/download/iphone"" r...",RT @ObamaFoundation: This week—50 years since ...,False,"{'id': 813286, 'id_str': '813286', 'name': 'Ba...",1,0,0.512344,0.000000,[]
4,4,NaN,NaN,Sun Mar 25 15:34:31 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,226302,False,NaN,977931743880761344,...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Incredible to have a Chicago team in the Final...,True,"{'id': 813286, 'id_str': '813286', 'name': 'Ba...",1,226302,0.728896,164950.595175,[]


In [59]:
def hash_tag_sent_build(df):
    hashtags = {}
    hts = df.hashtags.values
    pos_proba = df.pos_proba.values
    classify_rat = 0
    for i in tqdm(range(len(hts))):
        tmp_tag = list(eval(hts[i]))
       
        if len(tmp_tag) > 0:
            classify_rat +=1
            for j in tmp_tag:
                if j not in hashtags:
                    hashtags[j] = {0:0, 1:1, 'Count':0}
                hashtags[j][0] += (1- pos_proba[i])
                hashtags[j][1] += pos_proba[i]
                hashtags[j]['Count']+=1
    ret_df =  pd.DataFrame.from_dict(hashtags, orient='index').sort_values("Count", ascending=False)
    ret_df['pos'] = ret_df[1] / ( ret_df[0] +  ret_df[1])
    ret_df['neg'] = ret_df[0]/( ret_df[0] +  ret_df[1])
    return ret_df.loc[ret_df["Count"] > 1]

In [60]:
ht_df = hash_tag_sent_build(df)

In [61]:
ht_df.head()

,0,1,Count,pos,neg
ActOnClimate,99.714714,200.285286,299,0.667618,0.332382
DoYourJob,69.825085,81.174915,150,0.537582,0.462418
SOTU,50.936281,75.063719,125,0.595744,0.404256
GetCovered,33.633397,86.366603,119,0.719722,0.280278
Obamacare,27.654613,52.345387,79,0.654317,0.345683


In [62]:
ht_df

,0,1,Count,pos,neg
ActOnClimate,99.714714,200.285286,299,0.667618,0.332382
DoYourJob,69.825085,81.174915,150,0.537582,0.462418
SOTU,50.936281,75.063719,125,0.595744,0.404256
GetCovered,33.633397,86.366603,119,0.719722,0.280278
Obamacare,27.654613,52.345387,79,0.654317,0.345683
ImmigrationAction,27.796221,43.203779,70,0.608504,0.391496
IranDeal,19.795291,40.204709,59,0.670078,0.329922
SCOTUS,17.205233,29.794767,46,0.633931,0.366069
RaiseTheWage,13.596973,28.403027,41,0.676263,0.323737
LeadOnTrade,11.556266,26.443734,37,0.695888,0.304112


In [63]:
xs = OrdinalScale()
ys = LinearScale()

bar = Bars(x=ht_df.index.values[:10], y=ht_df.pos.values[:10], scales={'x': xs, 'y': ys}, padding=0.2, type='grouped')
xax = Axis(scale=xs)
yax = Axis(scale=ys, orientation='vertical', tick_format='0.0%', grid_lines='solid')
fig = Figure(marks=[bar], axes=[xax, yax], animation_duration=1000, )
fig.layout.width = '1200px'
fig

Figure(animation_duration=1000, axes=[Axis(scale=OrdinalScale()), Axis(orientation='vertical', scale=LinearSca…